<a href="https://colab.research.google.com/github/Tb-4-4/SummerAnalytics/blob/main/Dynamic_Pricing_for_Urban_Parking_Lots_Model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports & Installs**

In [1]:
!pip install pathway bokeh --quiet # This cell may take a few seconds to execute.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import pathway as pw
import panel as pn
import os

import bokeh.plotting
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Legend, HoverTool
from bokeh.palettes import Category20
from bokeh.transform import linear_cmap

# **Dataset Preprocessing, Encoding & Normalization**


In [3]:
urban_lots = pd.read_csv("/content/dataset.csv") # https://drive.google.com/file/d/1RqHF3zphAFOtYZgReDJUxEFweOiVAxqP/view?usp=drive_link
urban_lots

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18363,18363,Shopping,1920,26.150504,91.733531,1517,truck,average,6,0,19-12-2016,14:30:00
18364,18364,Shopping,1920,26.150504,91.733531,1487,car,low,3,0,19-12-2016,15:03:00
18365,18365,Shopping,1920,26.150504,91.733531,1432,cycle,low,3,0,19-12-2016,15:29:00
18366,18366,Shopping,1920,26.150504,91.733531,1321,car,low,2,0,19-12-2016,16:03:00


In [4]:
urban_lots['Timestamp'] = pd.to_datetime(urban_lots['LastUpdatedDate'] + ' ' + urban_lots['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
urban_lots = urban_lots.sort_values('Timestamp').reset_index(drop=True)
urban_lots

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00


In [5]:
traffic_map = {"low": 0, "average": 0.5, "high": 1}

In [6]:
urban_lots["TrafficLevel"] = urban_lots["TrafficConditionNearby"].map(traffic_map)

In [7]:
vehicle_weight = {"cycle": 0.2, "bike": 0.5, "car": 1.0, "truck": 1.2}

In [8]:
urban_lots["VehicleTypeWeight"] = urban_lots["VehicleType"].map(vehicle_weight)

In [9]:
q97 = urban_lots["QueueLength"].quantile(0.97)
q97

np.float64(10.0)

In [10]:
urban_lots["NormalizedQueue"] = urban_lots["QueueLength"] / q97
urban_lots["NormalizedQueue"] = urban_lots["NormalizedQueue"].clip(upper=1.0)

In [11]:
urban_lots

,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp,TrafficLevel,VehicleTypeWeight,NormalizedQueue
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.0,1.0,0.1
1,5248,BHMNCPHST01,1200,26.140014,91.731000,237,bike,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.0,0.5,0.2
2,3936,BHMMBMMBX01,687,20.000035,78.000003,264,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.0,1.0,0.2
3,6560,BHMNCPNST01,485,26.140048,91.730972,249,car,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.0,1.0,0.2
4,17056,Shopping,1920,26.150504,91.733531,614,cycle,low,2,0,04-10-2016,07:59:00,2016-10-04 07:59:00,0.0,0.2,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18363,3935,BHMEURBRD01,470,26.149020,91.739503,373,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.0,1.0,0.2
18364,2623,BHMBCCTHL01,387,26.144495,91.736205,387,car,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.0,1.0,0.2
18365,1311,BHMBCCMKT01,577,26.144536,91.736172,193,cycle,low,2,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.0,0.2,0.2
18366,17055,Others-CCCPS98,3103,26.147500,91.727978,1671,car,low,3,0,19-12-2016,16:30:00,2016-12-19 16:30:00,0.0,1.0,0.3


# **Processing Lots Individually**

In [12]:
os.makedirs("lots_csv", exist_ok=True)

lot_files = dict()

for system_code, group in urban_lots.groupby("SystemCodeNumber"):
    filename = f"lots_csv/{system_code}.csv"
    group.to_csv(filename, index=False)
    lot_files[system_code] = filename
    print(f"Saved {filename} with {len(group)} records")

Saved lots_csv/BHMBCCMKT01.csv with 1312 records
Saved lots_csv/BHMBCCTHL01.csv with 1312 records
Saved lots_csv/BHMEURBRD01.csv with 1312 records
Saved lots_csv/BHMMBMMBX01.csv with 1312 records
Saved lots_csv/BHMNCPHST01.csv with 1312 records
Saved lots_csv/BHMNCPNST01.csv with 1312 records
Saved lots_csv/Broad Street.csv with 1312 records
Saved lots_csv/Others-CCCPS105a.csv with 1312 records
Saved lots_csv/Others-CCCPS119a.csv with 1312 records
Saved lots_csv/Others-CCCPS135a.csv with 1312 records
Saved lots_csv/Others-CCCPS202.csv with 1312 records
Saved lots_csv/Others-CCCPS8.csv with 1312 records
Saved lots_csv/Others-CCCPS98.csv with 1312 records
Saved lots_csv/Shopping.csv with 1312 records


In [13]:
lot_files

{'BHMBCCMKT01': 'lots_csv/BHMBCCMKT01.csv',
 'BHMBCCTHL01': 'lots_csv/BHMBCCTHL01.csv',
 'BHMEURBRD01': 'lots_csv/BHMEURBRD01.csv',
 'BHMMBMMBX01': 'lots_csv/BHMMBMMBX01.csv',
 'BHMNCPHST01': 'lots_csv/BHMNCPHST01.csv',
 'BHMNCPNST01': 'lots_csv/BHMNCPNST01.csv',
 'Broad Street': 'lots_csv/Broad Street.csv',
 'Others-CCCPS105a': 'lots_csv/Others-CCCPS105a.csv',
 'Others-CCCPS119a': 'lots_csv/Others-CCCPS119a.csv',
 'Others-CCCPS135a': 'lots_csv/Others-CCCPS135a.csv',
 'Others-CCCPS202': 'lots_csv/Others-CCCPS202.csv',
 'Others-CCCPS8': 'lots_csv/Others-CCCPS8.csv',
 'Others-CCCPS98': 'lots_csv/Others-CCCPS98.csv',
 'Shopping': 'lots_csv/Shopping.csv'}

In [14]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    NormalizedQueue: float
    TrafficLevel: float
    VehicleTypeWeight: float
    IsSpecialDay: int

In [15]:
def make_plotter(lot_id):
    def price_plotter(source):
        import bokeh.plotting
        fig = bokeh.plotting.figure(
            height=400, width=800,
            title=f"Daily Price - {lot_id}",
            x_axis_type="datetime"
        )
        fig.line("t", "price", source=source, line_width=2, color="navy")
        fig.scatter("t", "price", source=source, marker="circle", size=5, color="red")
        return fig
    return price_plotter

In [21]:
fmt = "%Y-%m-%d %H:%M:%S"
figures = []
window_tables = {}

alpha = 0.8
beta = 0.7
gamma = 0.5
delta = 0.3
epsilon = 0.1

_lambda = 0.5

base_price = 10.0

for lot_id, file_path in lot_files.items():
    raw_data = pw.demo.replay_csv(file_path, schema=ParkingSchema, input_rate=100)

    data_with_time = raw_data.with_columns(
        t = raw_data.Timestamp.dt.strptime(fmt),
        day = raw_data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
    )

    delta_window = (
        data_with_time.windowby(
            pw.this.t,
            instance=pw.this.day,
            window=pw.temporal.tumbling(datetime.timedelta(days=1)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t = pw.this._pw_window_end,
            avg_occ = pw.reducers.avg(pw.this.Occupancy),
            cap = pw.reducers.max(pw.this.Capacity),
            avg_queue_norm = pw.reducers.avg(pw.this.NormalizedQueue),
            avg_traffic = pw.reducers.avg(pw.this.TrafficLevel),
            avg_vehicle_weight = pw.reducers.avg(pw.this.VehicleTypeWeight),
            is_special_day = pw.reducers.max(pw.this.IsSpecialDay)
        )

        .with_columns(
            demand = (
                alpha * (pw.this.avg_occ / pw.this.cap) +
                beta * pw.this.avg_queue_norm -
                gamma * pw.this.avg_traffic +
                delta * pw.this.is_special_day +
                epsilon * pw.this.avg_vehicle_weight
            )
        )

        .with_columns(
            norm_demand = pw.apply(lambda x: max(min(x, 1.0), -1.0), pw.this.demand)
        )

        .with_columns(
            raw_price = pw.apply(lambda d: base_price * (1 + _lambda * d), pw.this.norm_demand)
        )

        .with_columns(
            price = pw.apply(lambda p: max(min(p, 2.0 * base_price), 0.5 * base_price), pw.this.raw_price)
        )
    )

    window_tables[lot_id] = delta_window
    viz = delta_window.plot(make_plotter(lot_id), sorting_col="t")
    figures.append(pn.Column(viz))

In [22]:
len(window_tables)

14

# **Lot-wise Demand-based Price Prediction**


In [23]:
pn.Column(*figures).servable()

Column
    [0] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [1] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [2] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [3] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [4] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [5] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [6] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [7] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [8] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [9] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [10] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [11] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [12] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)
    [13] Column
        [0] Column
            [0] Row
                [0] Markdown(str)
                [1] TooltipIcon(value='Table depends o...)
            [1] Bokeh(figure)

In [24]:
%%capture --no-display
pw.run()

Output()